In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

import requests 
from bs4 import BeautifulSoup 
from lxml import html

In [2]:
#d1 = pd.read_csv("d1.csv")

In [3]:
#d1.head()

In [4]:
import mechanize
import http.cookiejar as cookielib

In [76]:
#https://stackoverflow.com/questions/20039643/how-to-scrape-a-website-that-requires-login-first-with-python
br = mechanize.Browser()

# Cookie Jar
cj = cookielib.LWPCookieJar()
br.set_cookiejar(cj)
br.set_handle_equiv(True)
br.set_handle_gzip(True)
br.set_handle_redirect(True)
br.set_handle_referer(True)
br.set_handle_robots(False)
br.set_handle_refresh(mechanize._http.HTTPRefreshProcessor(), max_time=1)

br.addheaders = [('User-agent', 'Chrome')]
br.open('http://dashboard.fsnau.org/index/auth')

<response_seek_wrapper at 0x14f7afb7208 whose wrapped object = <closeable_response at 0x14f7b0e5fd0 whose fp = <mechanize._gzip.UnzipWrapper object at 0x0000014F7B0E5E80>>>

In [77]:
for f in br.forms():
    print(f)

<post http://dashboard.fsnau.org/index/login application/x-www-form-urlencoded
  <TextControl(username=)>
  <PasswordControl(password=)>
  <SubmitControl(<None>=Login) (readonly)>>


In [78]:
br.select_form(nr=0)
br.form['username'] = 'prabhnoor1997ronaldo@gmail.com'
br.form['password'] = 'passwordis1234'
br.submit()

<response_seek_wrapper at 0x14f7b1c1780 whose wrapped object = <closeable_response at 0x14f7b1c76d8 whose fp = <mechanize._gzip.UnzipWrapper object at 0x0000014F7B1C7240>>>

In [79]:
url = "http://dashboard.fsnau.org/summary/ts/2?district=2&start=01-Jan-2016&end=01-Sep-2019"
soup = BeautifulSoup(br.open(url).read(), 'html.parser') 

In [80]:
print(soup.find_all('h3'))
print("***")
print("LOCATION: "+soup.find_all('h3')[1].text)

[<h3 style="margin-top:0.8em !important"><a href="http://dashboard.fsnau.org/dashboard/index">EW-EA Dashboard</a> / Districts / Borama </h3>, <h3>Awdal / Borama</h3>]
***
LOCATION: Awdal / Borama


In [81]:
#soup.find('table', attrs={'class':'table table-striped responsive-table'}).find('tbody').find_all('tr')[1].find_all('td')

In [82]:
data_dict = {}

In [83]:
for region_num in range(1,75):
    url = "http://dashboard.fsnau.org/summary/ts/2?district="+str(region_num)+"&start=01-Jan-2016&end=01-Sep-2019"
    soup = BeautifulSoup(br.open(url).read(), 'html.parser')
    region = soup.find_all('h3')[1].text.lower().replace(" / ","/").replace(" ","_")
    print("Scraping for region : %s .........."% (region))
    data_dict[region] = {}
    n_cols = len(soup.find('table', attrs={'class':'table table-striped responsive-table'}).find('tbody').find_all('tr'))
    
    for i in range(0, n_cols,1):
        child_items = soup.find('table', attrs={'class':'table table-striped responsive-table'}).find('tbody').find_all('tr')[i].find_all('td')
        col_name = child_items[1].text.lower().replace(" ","_")
        data_dict[region][col_name] = []
        for child in child_items[3:-1]:
            danger_level = child['class'][0]
            date = child['title'].split(" ")[0]
            value = child.text.strip()
            data_dict[region][col_name].append([date, danger_level, value])

Scraping for region : banadir/mogadishu ..........
Scraping for region : awdal/borama ..........
Scraping for region : awdal/baki ..........
Scraping for region : awdal/lughaye ..........
Scraping for region : awdal/zeylac ..........
Scraping for region : woqooyi_galbeed/hargeysa ..........
Scraping for region : woqooyi_galbeed/berbera ..........
Scraping for region : woqooyi_galbeed/gebiley ..........
Scraping for region : togdheer/burco ..........
Scraping for region : togdheer/buuhoodle ..........
Scraping for region : togdheer/owdweyne ..........
Scraping for region : togdheer/sheikh ..........
Scraping for region : sool/laas_caanood ..........
Scraping for region : sool/caynabo ..........
Scraping for region : sool/taleex ..........
Scraping for region : sool/xudun ..........
Scraping for region : sanaag/ceerigaabo ..........
Scraping for region : sanaag/ceel_afweyn ..........
Scraping for region : sanaag/laasqoray ..........
Scraping for region : bari/bossaso ..........
Scraping 

In [86]:
len(data_dict.keys())

74

In [96]:
temp = np.array([(len(data_dict[key]),key) for key in data_dict.keys()])

In [106]:
max_col_region = 'gedo/baardheere'

In [109]:
print(data_dict[max_col_region].keys())

dict_keys(['climate/rainfall', 'climate/vegetation_cover_(ndvi)', 'climate/price_of_water', 'climate/river_levels/floods', 'population_movement/displacement_(arrivals)', 'population_movement/displacement_(departures)', 'nutrition/new_admissions_(gam)', 'health/measles_cases', 'health/awd/cholera_cases', 'health/malaria_cases', 'health/awd/cholera_deaths', 'market/maize_prices', 'market/sorghum_prices', 'market/local_goat_prices', 'market/wage_labor', 'market/tot_(wage_to_cereals)', 'market/tot_(goat_to_cereals)', 'market/red_rice_prices', 'market/cost_of_minimum_basket_(cmb)'])


In [112]:
for region in data_dict.keys():
    if set(data_dict[region].keys()).issubset(data_dict[max_col_region].keys()):
        pass
    else:
        print(region)

In [170]:
data_cols= list(data_dict[max_col_region].keys())

In [172]:
data_cols.extend(['place'])
data_cols.extend(['date'])

In [173]:
data_cols

['climate/rainfall',
 'climate/vegetation_cover_(ndvi)',
 'climate/price_of_water',
 'climate/river_levels/floods',
 'population_movement/displacement_(arrivals)',
 'population_movement/displacement_(departures)',
 'nutrition/new_admissions_(gam)',
 'health/measles_cases',
 'health/awd/cholera_cases',
 'health/malaria_cases',
 'health/awd/cholera_deaths',
 'market/maize_prices',
 'market/sorghum_prices',
 'market/local_goat_prices',
 'market/wage_labor',
 'market/tot_(wage_to_cereals)',
 'market/tot_(goat_to_cereals)',
 'market/red_rice_prices',
 'market/cost_of_minimum_basket_(cmb)',
 'place',
 'date']

In [157]:
num_cols = len(data_dict[max_col_region].keys())

In [227]:
#data_list = [[]]*num_cols

In [183]:
data_cols.index(list(data_dict[region].keys())[0])

0

In [198]:
len(data_cols)

21

In [228]:
#data_cols.index(list(data_dict[region].keys())[3])

In [238]:
len(data_dict[region]['climate/rainfall'])

44

In [284]:
regions = list(data_dict.keys())

In [285]:
data_list = []
for i in range(0,num_cols):
    data_list.append([])
    
for region in regions:
    print("Processing Region %s >......."% (region))
    current_columns = list(data_dict[region].keys())
    for index, col in enumerate(data_cols[:-2]):
        if col in current_columns:
            for values_list in data_dict[region][col]:
                data_list[index].append(values_list[2])
        else:
            data_list[index].extend(['No Data']*44)

Processing Region banadir/mogadishu >.......
Processing Region awdal/borama >.......
Processing Region awdal/baki >.......
Processing Region awdal/lughaye >.......
Processing Region awdal/zeylac >.......
Processing Region woqooyi_galbeed/hargeysa >.......
Processing Region woqooyi_galbeed/berbera >.......
Processing Region woqooyi_galbeed/gebiley >.......
Processing Region togdheer/burco >.......
Processing Region togdheer/buuhoodle >.......
Processing Region togdheer/owdweyne >.......
Processing Region togdheer/sheikh >.......
Processing Region sool/laas_caanood >.......
Processing Region sool/caynabo >.......
Processing Region sool/taleex >.......
Processing Region sool/xudun >.......
Processing Region sanaag/ceerigaabo >.......
Processing Region sanaag/ceel_afweyn >.......
Processing Region sanaag/laasqoray >.......
Processing Region bari/bossaso >.......
Processing Region bari/bandarbeyla >.......
Processing Region bari/caluula >.......
Processing Region bari/iskushuban >.......
Pr

In [286]:
print(len(data_list))
print(len(data_list[0]))

19
3256


In [287]:
data_cols

['climate/rainfall',
 'climate/vegetation_cover_(ndvi)',
 'climate/price_of_water',
 'climate/river_levels/floods',
 'population_movement/displacement_(arrivals)',
 'population_movement/displacement_(departures)',
 'nutrition/new_admissions_(gam)',
 'health/measles_cases',
 'health/awd/cholera_cases',
 'health/malaria_cases',
 'health/awd/cholera_deaths',
 'market/maize_prices',
 'market/sorghum_prices',
 'market/local_goat_prices',
 'market/wage_labor',
 'market/tot_(wage_to_cereals)',
 'market/tot_(goat_to_cereals)',
 'market/red_rice_prices',
 'market/cost_of_minimum_basket_(cmb)',
 'place',
 'date']

In [288]:
import pandas as pd

In [289]:
fansu_data = pd.DataFrame(columns=data_cols[-2:]+data_cols[:-2])

In [290]:
for ind, col in enumerate(data_cols[:-2]):
    fansu_data[col] = data_list[ind]

In [291]:
solo_dates = [iterator[0] for iterator in data_dict['juba_hoose/jamaame']['climate/rainfall']]

In [292]:
fansu_data['date'] = solo_dates*74

In [304]:
repeated_regions = []
for r in regions:
    repeated_regions.extend([r]*44)
len(repeated_regions)

3256

In [305]:
fansu_data['place'] = repeated_regions

In [306]:
fansu_data

,place,date,climate/rainfall,climate/vegetation_cover_(ndvi),climate/price_of_water,climate/river_levels/floods,population_movement/displacement_(arrivals),population_movement/displacement_(departures),nutrition/new_admissions_(gam),health/measles_cases,...,health/malaria_cases,health/awd/cholera_deaths,market/maize_prices,market/sorghum_prices,market/local_goat_prices,market/wage_labor,market/tot_(wage_to_cereals),market/tot_(goat_to_cereals),market/red_rice_prices,market/cost_of_minimum_basket_(cmb)
0,banadir/mogadishu,Jan-2016,No Data,No Data,No Data,No Data,574,148,-,39,...,316,-,6500,6000,1600000,167500,26,246,12000,2024750
1,banadir/mogadishu,Feb-2016,No Data,No Data,No Data,No Data,708,32,2511,39,...,369,-,6400,6000,1600000,164000,26,250,12000,2028916.67
2,banadir/mogadishu,Mar-2016,No Data,No Data,No Data,No Data,656,16,5075,29,...,411,-,6425,5800,1650000,167500,26,257,12000,1960525
3,banadir/mogadishu,Apr-2016,No Data,No Data,No Data,No Data,498,40,4536,21,...,332,-,7260,6080,1840000,160000,22,253,12000,2033733.75
4,banadir/mogadishu,May-2016,No Data,No Data,No Data,No Data,716,22,2843,22,...,303,-,8960,6080,1900000,160000,18,212,12000,2065682.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3251,juba_hoose/jamaame,Apr-2019,24.02,0.29,10000,No Data,493,802,-,-,...,12,No Data,5500,No Data,825000,100000,18,150,16000,1855875
3252,juba_hoose/jamaame,May-2019,139.11,0.33,10000,No Data,777,1217,-,-,...,31,No Data,6900,No Data,825000,100000,14,120,16000,1904375
3253,juba_hoose/jamaame,Jun-2019,21.58,0.44,10000,No Data,802,1208,-,-,...,18,No Data,7000,No Data,800000,100000,14,114,16000,1960000
3254,juba_hoose/jamaame,Jul-2019,12.17,0.46,1000,No Data,910,1516,-,3,...,41,No Data,9000,No Data,800000,120000,13,89,15000,1951575
